In [9]:
from PDR.modules.DataLoader import *
from PDR.modules.Walker import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import plotly.graph_objects as go

In [10]:
sensor_df = pd.read_csv("data/200709_mag2.csv", names=['experiment', 'time', 'accx', 'accy', 'accz', 'gyrox', 'gyroy',
                                                               'gyroz', 'magx', 'magy', 'magz', 'rot0', 'rot1', 'rot2', 'rot3', 'game0', 'game1', 'game2', 'game3', 'light', "x_uncalib", "y_uncalib","z_uncalib","x_bias","y_bias","z_bias"])
sensor_df = sensor_df[['time','magx', 'magy', 'magz',"x_uncalib", "y_uncalib","z_uncalib","x_bias","y_bias","z_bias"]]
sensor_df.head()

,time,magx,magy,magz,x_uncalib,y_uncalib,z_uncalib,x_bias,y_bias,z_bias
0,1594284734091,-12.141861,-17.107270,-49.687020,2.114685,-6.347025,-147.17262,14.256546,10.760244,-97.4856
1,1594284734131,-14.444616,-14.504515,-48.541817,-0.188070,-3.744270,-146.02742,14.256546,10.760244,-97.4856
2,1594284734152,-12.157686,-17.844818,-48.949165,2.098860,-7.084575,-146.43477,14.256546,10.760244,-97.4856
3,1594284734172,-12.758466,-16.240600,-49.200706,1.498080,-5.480355,-146.68631,14.256546,10.760244,-97.4856
4,1594284734191,-15.229385,-14.230120,-47.314095,-0.972840,-3.469875,-144.79970,14.256546,10.760244,-97.4856


In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=sensor_df['time'], 
    y=sensor_df['magx'],
    name="mag"))
fig.add_trace(go.Scatter(
    x=sensor_df['time'], 
    y=sensor_df['x_uncalib'],
    name="uncailb"))
fig.add_trace(go.Scatter(
    x=sensor_df['time'], 
    y=sensor_df['x_bias'],
    name="bias"))
fig.add_trace(go.Scatter(
    x=sensor_df['time'], 
    y=sensor_df['x_uncalib'] - sensor_df['x_bias'],
    name="calib"))


In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=sensor_df['time'], 
    y=sensor_df['magy'],
    name="mag"))
fig.add_trace(go.Scatter(
    x=sensor_df['time'], 
    y=sensor_df['y_uncalib'],
    name="uncailb"))
fig.add_trace(go.Scatter(
    x=sensor_df['time'], 
    y=sensor_df['y_bias'],
    name="bias"))
fig.add_trace(go.Scatter(
    x=sensor_df['time'], 
    y=sensor_df['y_uncalib'] - sensor_df['y_bias'],
    name="calib"))


In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=sensor_df['time'], 
    y=sensor_df['magz'],
    name="mag"))
fig.add_trace(go.Scatter(
    x=sensor_df['time'], 
    y=sensor_df['z_uncalib'],
    name="uncailb"))
fig.add_trace(go.Scatter(
    x=sensor_df['time'], 
    y=sensor_df['z_bias'],
    name="bias"))
fig.add_trace(go.Scatter(
    x=sensor_df['time'], 
    y=sensor_df['z_uncalib'] - sensor_df['z_bias'],
    name="calib"))
